In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
!pip install pypdf -q

In [3]:
!pip install docx2txt -q

In [4]:
!pip install wikipedia -q

In [2]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)
    
    if extension == ".pdf":
        from langchain.document_loaders import PyPDFLoader
        print(f"Loading {file}")
        loader = PyPDFLoader(file)
    elif extension == ".docx":
        from langchain.document_loaders import Docx2txtLoader
        print(f"Loading {file}")
        loader = Docx2txtLoader(file)
    else:
        print("Document format is not supported!")
        return None
    
    data = loader.load()
    return data

# Wikipedia
def load_from_wikipedia(query, lang="en", load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [3]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks

In [4]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model("text-embedding-ada-002")
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f"Total tokens: {total_tokens}")
    print(f"Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}")

### Embedding and Uploading to a Vector Database (Pinecone)

In [5]:
def insert_or_fetch_embeddings(index_name):
    import pinecone
    from langchain.vectorstores import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings
    
    embeddings = OpenAIEmbeddings()
    
    pinecone.init(api_key=os.environ.get("PINECONE_API_KEY"), environment=os.environ.get("PINECONE_ENV"))
    
    if index_name in pinecone.list_indexes():
        print(f"Index {index_name} already exists. Loading embeddings ...", end="")
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
    else:
        print(f"Creating index {index_name} and embeddings ...", end="")
        pinecone.create_index(index_name, dimension=1536, metric="cosine")
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print("Ok")
    
    return vector_store

In [6]:
def delete_pinecone_index(index_name="all"):
    import pinecone
    pinecone.init(api_key=os.environ.get("PINECONE_API_KEY"), environment=os.environ.get("PINECONE_ENV"))
    
    if index_name == "all":
        indexes = pinecone.list_indexes()
        print("Deleting all indexes ...")
        for index in indexes:
            pinecone.delete_index(index)
        print("Ok")
    else:
        print(f"Deleting index {index_name} ...", end="")
        pinecone.delete_index(index_name)
        print("Ok")        

### Asking and Getting Answers

In [7]:
def ask_and_get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=1)

    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k" : 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.run(q)
    
    return answer

def ask_with_memory(vector_store, question, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI
    
    llm = ChatOpenAI(temperature=1)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k' : 3})
    
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({'question' : question, 'chat_history' : chat_history})
    chat_history.append((question, result['answer']))
    
    return result, chat_history  

### Running Code

In [8]:
data = load_document("files/us_constitution.pdf")
#print(data[1].page_content)
#print(data[10].metadata)
print(f"You have {len(data)} pages in your data")
print(f"There are {len(data[20].page_content)} characters in the page")

Loading files/us_constitution.pdf
You have 41 pages in your data
There are 1137 characters in the page


In [9]:
#data = load_document("files/the_great_gatsby.docx")
#print(data[0].page_content)

In [10]:
#data = load_from_wikipedia("GPT-4", "de")
#print(data[0].page_content)

In [11]:
chunks = chunk_data(data)
print(len(chunks))
print(chunks[10].page_content)

190
Representatives
shall
chuse
their
Speaker
and
other
Of ficers;and
shall
have
the
sole
Power
of
Impeachment.
Section
3:
The
Senate
The
Senate
of
the
United
States
shall
be
composed
of
two
Senators
from
each
State,
chosen
by
the
Legislature
thereof,
for
six


In [12]:
print_embedding_cost(chunks)

Total tokens: 16711
Embedding Cost in USD: 0.006684


In [13]:
delete_pinecone_index()

C:\Users\germa\anaconda3\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Deleting all indexes ...
Ok


In [14]:
index_name = "askadocument"
vector_store = insert_or_fetch_embeddings(index_name)

Creating index askadocument and embeddings ...Ok


In [15]:
q = "What is the whole document about?"
answer = ask_and_get_answer(vector_store, q)
print(answer)

The document is about the United States Constitution. It outlines the purpose of the Constitution, which is to form a more perfect union, establish justice, ensure domestic tranquility, provide for the common defense, promote the general welfare, and secure the blessings of liberty. It also discusses the authority of the Constitution and the supremacy of treaties made under the authority of the United States. Additionally, it mentions the rights and privileges of citizens in each state.


In [33]:
import time
i = 1
print("Write Quit or Exit to quit")
while True:
    q = input(f"Question #{i}; ")
    i = i+1
    if q.lower() in ["quit", "exit"]:
        print("Quitting ... bye bye!")
        time.sleep(2)
        break
    
    answer = ask_and_get_answer(vector_store, q)
    print(f"\nAnswer: {answer}")
    print(f'\n {"-" * 50} \n')

Write Quit or Exit to quit
Question #1; How many amendments are in the US Constitution?

Answer: There are currently 27 amendments in the US Constitution.

 -------------------------------------------------- 

Question #2; Multiply that number by 2

Answer: To multiply the number 1,760 million by 2, the result would be 3,520 million.

 -------------------------------------------------- 

Question #3; quit
Quitting ... bye bye!


In [18]:
delete_pinecone_index()

Deleting all indexes ...
Ok


In [19]:
data = load_from_wikipedia("ChatGPT", "es")
chunks = chunk_data(data)
index_name = "chatgpt"
vector_store = insert_or_fetch_embeddings(index_name)

Creating index chatgpt and embeddings ...Ok


In [20]:
q = "Que es ChatGPT"
answer = ask_and_get_answer(vector_store, q)
print(answer)

ChatGPT es una aplicación de chatbot de inteligencia artificial desarrollada por OpenAI en 2022. Es un modelo de lenguaje pre-entrenado que se especializa en el diálogo. Puede interactuar con los usuarios y responder preguntas sobre diversos temas, incluyendo fenómenos de Internet y lenguajes de programación como Python. A diferencia de su predecesor, ChatGPT se ha diseñado para reducir respuestas dañinas y engañosas, proporcionando información más precisa y confiable.


In [21]:
# Asking with Memory
chat_history = []
question = "How many amendments are in the U.S. Constitution?"
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)

There are currently 27 amendments in the U.S. Constitution.
[('How many amendments are in the U.S. Constitution?', 'There are currently 27 amendments in the U.S. Constitution.')]


In [22]:
question = "Multiply that number by 2"
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)

The given information does not provide a specific number to multiply by 2. Therefore, I cannot provide the result of multiplying that number by 2.
[('How many amendments are in the U.S. Constitution?', 'There are currently 27 amendments in the U.S. Constitution.'), ('Multiply that number by 2', 'The given information does not provide a specific number to multiply by 2. Therefore, I cannot provide the result of multiplying that number by 2.')]
